In [184]:

import pandas as pd
import pandasql as pds
from pandasql import sqldf
import plotly.graph_objects as go
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 50)
pd.set_option('display.min_rows', 50)
import seaborn as sns
from dateutil.relativedelta import relativedelta
import random
import numpy as np

In [185]:
big_lot_table_8 = pd.read_csv(r"C:\Users\MichaelDiFelice\Documents\Sanofi\MM\DATA\Flatiron\Big LoT Table\Big LoT Table.csv")

def calculate_lookback(lot_df, reference_date):
    # Convert STARTDATE to datetime format
    lot_df['STARTDATE'] = pd.to_datetime(lot_df['STARTDATE'])

    # Filter the rows based on ISFIRSTTREATMENT
    first_treatment_data = lot_df[lot_df['ISFIRSTTREATMENT'] == 1]

    # Create a copy of the filtered data to avoid SettingWithCopyWarning
    first_treatment_data_copy = first_treatment_data.copy()

    # Calculate the exact difference in months between two dates
    def months_difference(start_date, end_date):
        delta = relativedelta(end_date, start_date)
        return delta.years * 12 + delta.months

    # Calculate the lookback in 12-month increments (up to 48 months)
    def lookback_periods(months):
        if months < 12:
            return "0-12 months"
        elif months < 24:
            return "12-24 months"
        elif months < 36:
            return "24-36 months"
        elif months < 48:
            return "36-48 months"
        else:
            return "more than 48 months"

    # Calculate the exact months since start for each row
    first_treatment_data_copy['exact_months_since_start'] = first_treatment_data_copy['STARTDATE'].apply(lambda x: months_difference(x, reference_date))

    # Apply the lookback_periods function to determine the lookback based on the exact month difference
    first_treatment_data_copy['exact_lookback'] = first_treatment_data_copy['exact_months_since_start'].apply(lookback_periods)

    data_final = first_treatment_data_copy[first_treatment_data['LINE_ZERO_FLAG']==0]
    
    return data_final



In [186]:
def bundle_regimen(row):
    # Line-based conditions
    if row['LineNumber'] == 1:
        # Transplant and regimen conditions within Line 1
        if row['TRANSPLANT_FLAG'] == 1 and row['REGIMEN'] == "DVRd":
            return "DVRd"
        elif row['TRANSPLANT_FLAG'] == 1 and row['REGIMEN'] == "DVTd":
            return "DVTd"
        elif row['TRANSPLANT_FLAG'] == 1 and row['REGIMEN'] == "VCd":
            return "VCd"
        elif row['TRANSPLANT_FLAG'] == 1 and row['REGIMEN'] in ["VRd", "VR", "Vd",]:
            return "VRd"
        elif row['TRANSPLANT_FLAG'] == 0 and row['REGIMEN'] in ['D mono','DKd','DKRd','DPd','DVd','DVMp','DVTd','Other D']:
            return "D-other"
        elif row['TRANSPLANT_FLAG'] == 0 and row['REGIMEN'] == 'DRd':
            return "DRd"
        elif row['TRANSPLANT_FLAG'] == 0 and row['REGIMEN'] == 'DVRd':
            return "DVRd"
        elif row['TRANSPLANT_FLAG'] == 0 and row['REGIMEN'] in ['Kd','Pd','Vd']:
            return 'Other doublet'
        elif row['TRANSPLANT_FLAG'] == 0 and row['REGIMEN'] in ['EPd','ERd','IRd','IsaKd','IsaPd','KPd','KRd','PCd','PVd','VCd','VMp','VTd']:
            return 'Other triplet'
        elif row['TRANSPLANT_FLAG'] == 0 and row['REGIMEN'] == 'Rd':
            return 'Rd'
        elif row['TRANSPLANT_FLAG'] == 0 and row['REGIMEN'] == 'VRd':
            return 'VRd' 

    elif row['LineNumber'] == 2:
        # Transplant and regimen conditions within Line 2
        if row['REGIMEN'] in ['D mono','DKRd','DVMp','DVRd','DVTd','Other D']:
            return "D-other"
        elif row['REGIMEN'] == "DKd":
            return "DKd"
        elif row['REGIMEN'] == "DPd":
            return "DPd"
        elif row['REGIMEN'] == "DRd":
            return "DRd"
        elif row['REGIMEN'] == "DVd":
            return "DVd"
        elif row['REGIMEN'] == "IsaKd":
            return "IsaKd"
        elif row['REGIMEN'] == "IsaPd":
            return "IsaPd"
        elif row['REGIMEN'] == 'Rd':
            return 'Rd'
        elif row["REGIMEN"] in ['ERd','IRd','KRd','VRd']:
            return 'R-Triplet'
    elif row['LineNumber'] == 3:
        # Transplant and regimen conditions within Line 2
        if row['REGIMEN'] in ['D mono','DKRd','DVMp','DVRd','DVTd','Other D']:
            return "D-other"
        elif row['REGIMEN'] == "DKd":
            return "DKd"
        elif row['REGIMEN'] == "DPd":
            return "DPd"
        elif row['REGIMEN'] == "DRd":
            return "DRd"
        elif row['REGIMEN'] == "DVd":
            return "DVd"
        elif row['REGIMEN'] == "IsaKd":
            return "IsaKd"
        elif row['REGIMEN'] == "IsaPd":
            return "IsaPd"
        elif row['REGIMEN'] == 'Rd':
            return 'Rd'
        elif row["REGIMEN"] in ['ERd','IRd','KRd','VRd']:
            return 'R-Triplet'


    # Default return value if none of the conditions are met
    return "Other"




In [187]:
def define_regimen_order(data):
    data_sorted = data.drop_duplicates(subset=['PATIENTID', 'LINENUMBER'])

    # Sort the data by PATIENTID and LINENUMBER to ensure the order
    data_sorted = data_sorted.sort_values(by=["PATIENTID", "LINENUMBER"])

    # Shift the REGIMEN column to get the next regimen for each patient and line
    data_sorted["NEXT_REGIMEN"] = data_sorted.groupby("PATIENTID")["REGIMEN"].shift(-1)
    data_sorted['PREVIOUS_REGIMEN'] = data_sorted.groupby("PATIENTID")["REGIMEN"].shift(1)
    # Fill NaN values in "NEXT_REGIMEN" with "No Advancement"
    data_sorted["NEXT_REGIMEN"].fillna("No Advancement", inplace=True)
    data_sorted["PREVIOUS_REGIMEN"].fillna("", inplace=True)


    # If NEXT_REGIMEN is 'No Advancement', set NEXT_LINENUMBER to the same as LINENUMBER
    # Otherwise, shift the LINENUMBER column to get the next line number for each patient
    data_sorted["NEXT_LINENUMBER"] = data_sorted.apply(
        lambda row: row["LINENUMBER"] if row["NEXT_REGIMEN"] == "No Advancement" else row["LINENUMBER"] + 1, axis=1
    )

    data_sorted["START_YEAR"] = data_sorted['exact_lookback']

    # Filter out rows where the next line is not consecutive (e.g., line 1 followed by line 3) or 'No Advancement'
    data_sorted = data_sorted[(data_sorted["LINENUMBER"] + 1 == data_sorted["NEXT_LINENUMBER"]) | (data_sorted["NEXT_REGIMEN"] == "No Advancement")]


    # Do the same steps above to bundled regimen
    data_sorted["NEXT_BUNDLE"] = data_sorted.groupby("PATIENTID")["BUNDLED_REGIMEN"].shift(-1)
    data_sorted['PREVIOUS_BUNDLE'] = data_sorted.groupby("PATIENTID")["BUNDLED_REGIMEN"].shift(1)
    # Fill NaN values in "NEXT_REGIMEN" with "No Advancement"
    data_sorted["NEXT_BUNDLE"].fillna("No Advancement", inplace=True)
    data_sorted["PREVIOUS_BUNDLE"].fillna("", inplace=True)

    data_sorted["NEXT_LINENUMBER"] = data_sorted.apply(
        lambda row: row["LINENUMBER"] if row["NEXT_BUNDLE"] == "No Advancement" else row["LINENUMBER"] + 1, axis=1
    )

    data_sorted["START_YEAR"] = data_sorted['exact_lookback']

    # Filter out rows where the next line is not consecutive (e.g., line 1 followed by line 3) or 'No Advancement'
    data_sorted = data_sorted[(data_sorted["LINENUMBER"] + 1 == data_sorted["NEXT_LINENUMBER"]) | (data_sorted["NEXT_BUNDLE"] == "No Advancement")]

    # For our current task, we only need certain columns
    data = data_sorted[["PATIENTID", "LINENUMBER", "REGIMEN","PREVIOUS_REGIMEN", "NEXT_REGIMEN","BUNDLED_REGIMEN",'PREVIOUS_BUNDLE','NEXT_BUNDLE', "LEN_FLAG", "CD38_FLAG", "CD38_EXPOSED_FLAG", "TRANSPLANT_FLAG", "START_YEAR","LEN_REFRACTORY_FLAG","NEXT_LINENUMBER","LineNumber"]]

    data = data[data['LINENUMBER'].isin([1, 2, 3, 4])]

    # Concatenate LINENUMBER to REGIMEN and NEXT_REGIMEN columns
    data['BUNDLED_REGIMEN'] = data['BUNDLED_REGIMEN'].astype(str) + data['LINENUMBER'].astype(str)
    data['NEXT_BUNDLE'] = data['NEXT_BUNDLE'].astype(str) + (data['LINENUMBER'] + 1).astype(str)
    data['PREVIOUS_BUNDLE'] = data['PREVIOUS_BUNDLE'].astype(str) + (data['LINENUMBER'] - 1).astype(str)
    data['PREVIOUS_BUNDLE'] = data['PREVIOUS_BUNDLE'].str.replace('0', '')

    # Concatenate LINENUMBER to REGIMEN and NEXT_REGIMEN columns
    data['REGIMEN'] = data['REGIMEN'].astype(str) + data['LINENUMBER'].astype(str)
    data['NEXT_REGIMEN'] = data['NEXT_REGIMEN'].astype(str) + (data['LINENUMBER'] + 1).astype(str)
    data['PREVIOUS_REGIMEN'] = data['PREVIOUS_REGIMEN'].astype(str) + (data['LINENUMBER'] - 1).astype(str)
    data['PREVIOUS_REGIMEN'] = data['PREVIOUS_REGIMEN'].str.replace('0', '')

   
    # Turn the data into a cvs in the Modified Data folder    
    data.to_csv(r"C:\Users\MichaelDiFelice\Documents\Sanofi\Python\Dashboard\Data\Modified Data\modified_sankey.csv", index=False)




In [188]:
data = pd.read_csv(r"C:\Users\MichaelDiFelice\Documents\Sanofi\Python\Dashboard\Data\Raw Data\flatiron_newlot_final_v2.csv")

data['newline6'] = data['newline6'].astype(int)

data.head()

,Unnamed: 0,PatientID,LineName,LineNumber,IsMaintenanceTherapy,StartDate,EndDate,gap,Consolidation,Induction,IsInductionMaintenance,new IsMaintenance,first_ep,last_ep,Original LineNumber,newline,newline2,newline3,newline4,newline5,newline6
0,0,F0004757B960D,"Bortezomib,Dexamethasone,Lenalidomide",1.0,False,2018-12-31,2023-07-31,NaN,0.0,0,False,False,2018-12-31,2023-07-31,1,1,1.0,1.0,1.0,1.0,1
1,1,F000B7198BCE1,"Bortezomib,Dexamethasone,Lenalidomide",1.0,False,2018-07-17,2021-10-31,NaN,0.0,0,False,False,2018-07-17,2021-10-31,1,1,1.0,1.0,1.0,1.0,1
2,2,F000B7198BCE1,"Dexamethasone,Pomalidomide",2.0,False,2021-11-01,2022-06-06,1.0,0.0,0,False,False,2021-11-01,2022-06-06,2,2,2.0,2.0,2.0,2.0,2
3,3,F000B7198BCE1,"Bortezomib,Cyclophosphamide,Dexamethasone,Isat...",3.0,False,2022-06-07,2022-07-05,1.0,0.0,0,False,False,2022-06-07,2022-06-21,3,3,3.0,3.0,3.0,3.0,3
4,4,F000C4086FB76,Dexamethasone,1.0,False,2020-09-10,2020-09-17,NaN,0.0,0,False,False,2020-09-10,2020-09-10,1,1,1.0,1.0,1.0,1.0,1


In [197]:
merge_data = big_lot_table_8.merge(
    data[['PatientID', 'LineNumber', 'newline6', 'Consolidation', 'Induction', 'StartDate', 'EndDate']],
    left_on=['PATIENTID', 'STARTDATE','ENDDATE'],
    right_on=['PatientID', 'StartDate','EndDate'],
    how='left'
)

merge_data['MIN_STARTDATE'] = merge_data.groupby(['PATIENTID', 'LineNumber'])['STARTDATE'].transform('min')
merge_data['ISFIRSTTREATMENT'] = np.where(merge_data['STARTDATE'] == merge_data['MIN_STARTDATE'], 1, 0)
merge_data['CD38_FLAG'] = np.where(merge_data['LINENAME'].str.contains('Daratumumab') | merge_data['LINENAME'].str.contains('Isatuximab-Irfc'), 1, 0)
# merge_data['CD38_FLAG'] = np.where(merge_data['LINENAME'].str.contains('Isatuximab-Irfc'), 1, 0)



data_final = calculate_lookback(merge_data, pd.to_datetime('2023-07-31'))

data_final['LINENUMBER'] = data_final['LineNumber']
data_final = data_final.drop('PatientID', axis=1)
data_final['LINENUMBER'] = np.where(data_final['LINENUMBER'] >= 4, 4, data_final['LINENUMBER'])

data_final['BUNDLED_REGIMEN'] = data_final.apply(bundle_regimen, axis=1)

define_regimen_order(data_final)

data_final.head()

,PATIENTID,LINENAME,LINENUMBER,LINESETTING,ISMAINTENANCETHERAPY,ENHANCEDCOHORT,STARTDATE,ENDDATE,REGIMEN,COMBINEDLINE,ISFIRSTTREATMENT,FIRST_CD38_DATE,CD38_EXPOSED_FLAG,TRANSPLANT_FLAG,ENHANCED_LINENUMBER,FIRST_LEN_DATE,LEN_FLAG,LINE_ZERO_FLAG,LEN_REFRACTORY_FLAG,CD38_FLAG,LEN_REFRACTORY_BY_NAME_FLAG,LineNumber,newline6,Consolidation,Induction,StartDate,EndDate,MIN_STARTDATE,exact_months_since_start,exact_lookback,BUNDLED_REGIMEN
0,F0004757B960D,"Bortezomib,Dexamethasone,Lenalidomide",1.0,\N,False,MultipleMyeloma,2018-12-31,2023-07-31,VRd,"Bortezomib,Dexamethasone,Lenalidomide",1,\N,0,0,1,2018-12-31,0,0,0,0,0,1.0,1.0,0.0,0.0,2018-12-31,2023-07-31,2018-12-31,55,more than 48 months,VRd
1,F000B7198BCE1,"Bortezomib,Dexamethasone,Lenalidomide",1.0,\N,False,MultipleMyeloma,2018-07-17,2021-10-31,VRd,"Bortezomib,Dexamethasone,Lenalidomide",1,2022-06-07,0,0,1,2018-07-17,0,0,0,0,0,1.0,1.0,0.0,0.0,2018-07-17,2021-10-31,2018-07-17,60,more than 48 months,VRd
2,F000B7198BCE1,"Dexamethasone,Pomalidomide",2.0,\N,False,MultipleMyeloma,2021-11-01,2022-06-06,Pd,"Dexamethasone,Pomalidomide",1,2022-06-07,0,0,2,2018-07-17,1,0,0,0,0,2.0,2.0,0.0,0.0,2021-11-01,2022-06-06,2021-11-01,20,12-24 months,Other
3,F000C4086FB76,Dexamethasone,1.0,\N,False,MultipleMyeloma,2020-09-10,2020-09-17,d mono,Dexamethasone,1,\N,0,0,1,\N,0,0,0,0,0,1.0,1.0,0.0,0.0,2020-09-10,2020-09-17,2020-09-10,34,24-36 months,Other
4,F000DC8ABFEB8,"Bortezomib,Dexamethasone,Lenalidomide",1.0,\N,False,MultipleMyeloma,2019-03-27,2021-01-25,VRd,"Bortezomib,Dexamethasone,Lenalidomide",1,2021-01-26,0,0,1,2019-03-27,0,0,0,0,0,1.0,1.0,0.0,0.0,2019-03-27,2021-01-25,2019-03-27,52,more than 48 months,VRd


In [190]:

185/253


0.7312252964426877

In [191]:
data_final = data_final.drop('StartDate', axis=1)
data_final = data_final.drop('EndDate', axis=1)
data_final = data_final.drop('LineNumber', axis=1)
data_final.head()


,PATIENTID,LINENAME,LINENUMBER,LINESETTING,ISMAINTENANCETHERAPY,ENHANCEDCOHORT,STARTDATE,ENDDATE,REGIMEN,COMBINEDLINE,ISFIRSTTREATMENT,FIRST_CD38_DATE,CD38_EXPOSED_FLAG,TRANSPLANT_FLAG,ENHANCED_LINENUMBER,FIRST_LEN_DATE,LEN_FLAG,LINE_ZERO_FLAG,LEN_REFRACTORY_FLAG,CD38_FLAG,LEN_REFRACTORY_BY_NAME_FLAG,newline6,Consolidation,Induction,MIN_STARTDATE,exact_months_since_start,exact_lookback,BUNDLED_REGIMEN
0,F0004757B960D,"Bortezomib,Dexamethasone,Lenalidomide",1.0,\N,False,MultipleMyeloma,2018-12-31,2023-07-31,VRd,"Bortezomib,Dexamethasone,Lenalidomide",1,\N,0,0,1,2018-12-31,0,0,0,0,0,1.0,0.0,0.0,2018-12-31,55,more than 48 months,VRd
1,F000B7198BCE1,"Bortezomib,Dexamethasone,Lenalidomide",1.0,\N,False,MultipleMyeloma,2018-07-17,2021-10-31,VRd,"Bortezomib,Dexamethasone,Lenalidomide",1,2022-06-07,0,0,1,2018-07-17,0,0,0,0,0,1.0,0.0,0.0,2018-07-17,60,more than 48 months,VRd
2,F000B7198BCE1,"Dexamethasone,Pomalidomide",2.0,\N,False,MultipleMyeloma,2021-11-01,2022-06-06,Pd,"Dexamethasone,Pomalidomide",1,2022-06-07,0,0,2,2018-07-17,1,0,0,0,0,2.0,0.0,0.0,2021-11-01,20,12-24 months,Other
3,F000C4086FB76,Dexamethasone,1.0,\N,False,MultipleMyeloma,2020-09-10,2020-09-17,d mono,Dexamethasone,1,\N,0,0,1,\N,0,0,0,0,0,1.0,0.0,0.0,2020-09-10,34,24-36 months,Other
4,F000DC8ABFEB8,"Bortezomib,Dexamethasone,Lenalidomide",1.0,\N,False,MultipleMyeloma,2019-03-27,2021-01-25,VRd,"Bortezomib,Dexamethasone,Lenalidomide",1,2021-01-26,0,0,1,2019-03-27,0,0,0,0,0,1.0,0.0,0.0,2019-03-27,52,more than 48 months,VRd


In [192]:
data_final.to_clipboard()

### Induction Sankey Creation

In [193]:
# Step 1: Filter Patients with Line 1 Transplant
transplant_patients = data_final[(data_final['COMBINEDLINE'].str.contains('Transplant', case=False)) & 
                                    (data_final['LINENUMBER'] == 1)]

# Extracting patients' IDs
transplant_patient_ids = transplant_patients['PATIENTID'].unique()

# Step 2: Identify Induction Regimen
induction_data = data_final[(data_final['PATIENTID'].isin(transplant_patient_ids)) & 
                               (data_final['ISFIRSTTREATMENT'] == 1) &
                               (data_final['LINENUMBER'] == 1)][['PATIENTID', 'REGIMEN', 'STARTDATE', 'ENDDATE']]

# Step 3: Identify Maintenance Therapy
maintenance_data = data_final[(data_final['PATIENTID'].isin(transplant_patient_ids)) & 
                                 (data_final['ISMAINTENANCETHERAPY'] == True)].sort_values('STARTDATE', ascending=False).drop_duplicates('PATIENTID')

# Step 4: Identify Line 2 Regimen with updated criteria
line2_data = data_final[(data_final['PATIENTID'].isin(transplant_patient_ids)) & 
                           (data_final['LINENUMBER'] == 2) & 
                           (data_final['ISFIRSTTREATMENT'] == 1)][['PATIENTID', 'REGIMEN', 'STARTDATE', 'ENDDATE']]

# Merge the data for a complete view
merged_data = induction_data.merge(maintenance_data[['PATIENTID', 'REGIMEN', 'STARTDATE', 'ENDDATE']], on='PATIENTID', how='left', suffixes=('_induction', '_maintenance')).merge(line2_data, on='PATIENTID', how='left', suffixes=('', '_line2'))

# Filter the dataset to the specified date range
filtered_data = merged_data[(merged_data['STARTDATE'] >= '2022-07-31') & (merged_data['STARTDATE'] <= '2023-07-31')]

# Extracting the CD38_FLAG for the patients in our filtered dataset
cd38_flags = data_final[data_final['PATIENTID'].isin(filtered_data['PATIENTID']) & 
                           (data_final['LINENUMBER'] == 2)][['PATIENTID', 'CD38_FLAG']]

# Merging the CD38_FLAG with our filtered dataset
filtered_data_with_cd38 = filtered_data.merge(cd38_flags, on='PATIENTID', how='left')

# Filtering the dataset to include only patients with a CD38 flag in Line 2
cd38_filtered_data_corrected = filtered_data_with_cd38[filtered_data_with_cd38['CD38_FLAG'] == 1]

# Filling NA values in the 'REGIMEN_maintenance' column with 'No Maintenance'
cd38_filtered_data_corrected['REGIMEN_maintenance'].fillna('No Maintenance', inplace=True)

# Drop duplicate rows based on PATIENTID
cd38_filtered_data= cd38_filtered_data_corrected.drop_duplicates(subset=['PATIENTID'])


C:\Users\MichaelDiFelice\AppData\Local\Temp\ipykernel_13212\2341165608.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [194]:
#---------------------------------------------------------Line 1 Code ----------------------------------------------------------------
#---------------------------------------------------------TOTAL PATIENTS----------------------------------------------------------------
line_1_query = """
WITH LineProgressionPatients AS (
    SELECT 
        PATIENTID, 
        CD38_FLAG,
        strftime('%Y%m', STARTDATE) AS TreatmentYear
    FROM data_final
    WHERE 
        LINENUMBER = 1
        AND LINE_ZERO_FLAG = 0
        AND strftime('%Y', STARTDATE) >= '2016'
        AND ISFIRSTTREATMENT = 1
)

SELECT 
    TreatmentYear,
    COUNT(DISTINCT CASE WHEN CD38_FLAG = 1 THEN PATIENTID END) AS CD38_Patients,
    COUNT(DISTINCT CASE WHEN CD38_FLAG = 0 THEN PATIENTID END) AS Non_CD38_Patients
FROM 
    LineProgressionPatients
GROUP BY 
    TreatmentYear
ORDER BY 
    TreatmentYear;
"""

#--------------------------------------------------------------------------CD38 PATIENTS--------------------------------------------------------------
line_2_CD_query = """
WITH RECURSIVE Calendar AS (
    SELECT 2016 AS TreatmentYear -- Start from 2016
    UNION ALL
    SELECT TreatmentYear + 1
    FROM Calendar
    WHERE TreatmentYear < 2023 -- Stop at 2023
),

Line_Progression_CD38_Patients AS (
    SELECT 
        PATIENTID,
        strftime('%Y', STARTDATE) AS TreatmentYear
    FROM 
        data_final
    WHERE 
        LINENUMBER = 1
        AND LINE_ZERO_FLAG = 0
        AND CD38_FLAG = 1
        AND ISFIRSTTREATMENT = 1
        AND strftime('%Y', STARTDATE) >= '2016'
),

NextLineFor_CD38_Patients AS (
    SELECT 
        l1.TreatmentYear,
        t.PATIENTID,
        t.CD38_FLAG AS NextLine_CD38_FLAG
    FROM 
        data_final t
    JOIN 
        Line_Progression_CD38_Patients l1 ON t.PATIENTID = l1.PATIENTID
    WHERE 
        t.LINENUMBER = 2
),

Line_Progression_CD38_Patients_Time AS (
    SELECT 
        PATIENTID,
        strftime('%Y', STARTDATE) AS TreatmentYear,
        STARTDATE AS FirstLineStartDate
    FROM 
        data_final
    WHERE 
        LINENUMBER = 1
        AND LINE_ZERO_FLAG = 0
        AND CD38_FLAG = 1
        AND ISFIRSTTREATMENT = 1
        AND strftime('%Y', STARTDATE) >= '2016'
),

NextLineFor_CD38_Patients_Time AS (
    SELECT 
        l2.TreatmentYear,
        t.PATIENTID,
        t.CD38_FLAG AS NextLine_CD38_FLAG,
        ((julianday(t.STARTDATE) - julianday(l2.FirstLineStartDate)) / 30) AS MonthsToNextLine
    FROM 
        data_final t
    JOIN 
        Line_Progression_CD38_Patients_Time l2 ON t.PATIENTID = l2.PATIENTID
    WHERE 
        t.LINENUMBER = 2
)

SELECT 
    c.TreatmentYear,
    COUNT(DISTINCT CASE WHEN n.NextLine_CD38_FLAG = 1 THEN n.PATIENTID END) AS To_CD38,
    COUNT(DISTINCT CASE WHEN n.NextLine_CD38_FLAG = 0 THEN n.PATIENTID END) AS To_Non_CD38,
    AVG(CASE WHEN nt.NextLine_CD38_FLAG = 1 THEN nt.MonthsToNextLine END) AS AvgMonths_To_CD38,
    AVG(CASE WHEN nt.NextLine_CD38_FLAG = 0 THEN nt.MonthsToNextLine END) AS AvgMonths_To_Non_CD38
FROM 
    Calendar c
LEFT JOIN 
    NextLineFor_CD38_Patients n ON c.TreatmentYear = n.TreatmentYear
LEFT JOIN 
    NextLineFor_CD38_Patients_Time nt ON c.TreatmentYear = nt.TreatmentYear
GROUP BY 
    c.TreatmentYear
ORDER BY 
    c.TreatmentYear;
"""
#--------------------------------------------------------------------------NON CD38 PATIENTS--------------------------------------------------------------
line_2_non_CD_query = """
WITH RECURSIVE Calendar AS (
    SELECT 2016 AS TreatmentYear -- Start from 2016
    UNION ALL
    SELECT TreatmentYear + 1
    FROM Calendar
    WHERE TreatmentYear < 2023 -- Stop at 2023
),

Line_Progression_Non_CD38_Patients AS (
    SELECT 
        PATIENTID,
        EXTRACT(YEAR FROM STARTDATE) AS TreatmentYear
    FROM 
        data_final
    WHERE 
        LINENUMBER = 1
        AND LINE_ZERO_FLAG = 0
        AND CD38_FLAG = 0
        AND ISFIRSTTREATMENT = 1
        AND EXTRACT(YEAR FROM STARTDATE) >= 2016
),

NextLineFor_Non_CD38_Patients AS (
    SELECT 
        l1.TreatmentYear,
        t.PATIENTID,
        t.CD38_FLAG AS NextLine_CD38_FLAG
    FROM 
        data_final t
    JOIN 
        Line_Progression_Non_CD38_Patients l1 ON t.PATIENTID = l1.PATIENTID
    WHERE 
        t.LINENUMBER = 2
),

Line_Progression_CD38_Patients_Time AS (
    SELECT 
        PATIENTID,
        EXTRACT(YEAR FROM STARTDATE) AS TreatmentYear,
        STARTDATE AS FirstLineStartDate
    FROM 
        data_final
    WHERE 
        LINENUMBER = 1
        AND LINE_ZERO_FLAG = 0
        AND CD38_FLAG = 0
        AND ISFIRSTTREATMENT = 1
        AND EXTRACT(YEAR FROM STARTDATE) >= 2016
),

NextLineFor_CD38_Patients_Time AS (
    SELECT 
        l2.TreatmentYear,
        t.PATIENTID,
        t.CD38_FLAG AS NextLine_CD38_FLAG,
        MONTHS_BETWEEN(t.STARTDATE, l2.FirstLineStartDate) AS MonthsToNextLine
    FROM 
        data_final t
    JOIN 
        Line_Progression_CD38_Patients_Time l2 ON t.PATIENTID = l2.PATIENTID
    WHERE 
        t.LINENUMBER = 2
)

SELECT 
    c.TreatmentYear,
    COUNT(DISTINCT CASE WHEN n.NextLine_CD38_FLAG = 1 THEN n.PATIENTID END) AS To_CD38,
    COUNT(DISTINCT CASE WHEN n.NextLine_CD38_FLAG = 0 THEN n.PATIENTID END) AS Remain_Non_CD38,
    AVG(CASE WHEN nt.NextLine_CD38_FLAG = 1 THEN nt.MonthsToNextLine END) AS AvgMonths_To_CD38,
    AVG(CASE WHEN nt.NextLine_CD38_FLAG = 0 THEN nt.MonthsToNextLine END) AS AvgMonths_To_Non_CD38
FROM 
    Calendar c
LEFT JOIN 
    NextLineFor_Non_CD38_Patients n ON c.TreatmentYear = n.TreatmentYear
LEFT JOIN 
    NextLineFor_CD38_Patients_Time nt ON c.TreatmentYear = nt.TreatmentYear
GROUP BY 
    c.TreatmentYear
ORDER BY 
    c.TreatmentYear;
"""

In [195]:
query1 = """
    SELECT 
        c.TreatmentYearMonth,
        COUNT(DISTINCT CASE WHEN d.LINENUMBER = 2 AND d.LINE_ZERO_FLAG = 0 AND d.CD38_FLAG = 1 AND d.ISFIRSTTREATMENT = 1 AND d.CD38_EXPOSED_FLAG = 0 THEN d.PATIENTID END) AS PatientCount
    FROM 
        calendar_df c
    LEFT JOIN 
        data_final d ON c.TreatmentYearMonth = strftime('%Y%m', d.STARTDATE)
    WHERE 
        strftime('%Y', d.STARTDATE) >= '2016'
    GROUP BY c.TreatmentYearMonth
"""
result1 = pds.sqldf(query1, locals())
result1.to_clipboard()

In [196]:
line_progression_df = result1

query2 = """
    SELECT 
        l1.TreatmentYearMonth,
        t.PATIENTID,
        t.CD38_FLAG AS NextLine_CD38_FLAG
    FROM 
        data_final t
    JOIN 
        line_progression_df l1 ON t.PATIENTID = l1.PATIENTID
    WHERE 
        t.LINENUMBER = 2
"""
result2 = pds.sqldf(query2, locals())
result2.head()

PandaSQLException: (sqlite3.OperationalError) no such column: l1.PATIENTID
[SQL: 
    SELECT 
        l1.TreatmentYearMonth,
        t.PATIENTID,
        t.CD38_FLAG AS NextLine_CD38_FLAG
    FROM 
        data_final t
    JOIN 
        line_progression_df l1 ON t.PATIENTID = l1.PATIENTID
    WHERE 
        t.LINENUMBER = 2
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
# Generate a DataFrame with a column for each month from 201601 to 202307
calendar_df = pd.DataFrame({'TreatmentYearMonth': pd.date_range(start='2016-01', end='2023-07', freq='M').strftime('%Y%m')})

# Make sure pandasql can access the new DataFrame
locals().update({'calendar_df': calendar_df})

query3 = """
SELECT 
    c.TreatmentYearMonth,
    COUNT(DISTINCT CASE WHEN r.NEXTLINE_CD38_FLAG = 1 THEN r.PATIENTID END) AS CD38_Patients,
    COUNT(DISTINCT CASE WHEN r.NEXTLINE_CD38_FLAG = 0 THEN r.PATIENTID END) AS Non_CD38_Patients
FROM 
    calendar_df c
LEFT JOIN 
    result2 r ON c.TreatmentYearMonth = r.TreatmentYearMonth
GROUP BY 
    c.TreatmentYearMonth
"""
result3 = pds.sqldf(query3, locals())
result3.to_clipboard()
result3

In [ ]:
query_first_second_dates = """
    SELECT 
        PATIENTID,
        MIN(CASE WHEN LINENUMBER = 1 THEN STARTDATE END) as FirstLineStartDate,
        MIN(CASE WHEN LINENUMBER = 2 THEN STARTDATE END) as SecondLineStartDate
    FROM 
        data_final
    WHERE 
        LINENUMBER IN (1, 2)
        AND LINE_ZERO_FLAG = 0
        AND ISFIRSTTREATMENT = 1
    GROUP BY 
        PATIENTID
"""
first_second_dates_df = pds.sqldf(query_first_second_dates, locals())
first_second_dates_df.head()


In [ ]:
result2['MonthsToNextLine'] = ((pd.to_datetime(result2['STARTDATE_LINE2']) - pd.to_datetime(result2['STARTDATE_LINE1'])).dt.days / 30).round()


In [ ]:
# Add a column to result2 that calculates the absolute difference in months between the STARTDATE of line 2 and the STARTDATE of line 1 for each patient
result2['MonthsToNextLine'] = (abs(pd.to_datetime(first_second_dates_df['FirstLineStartDate']) - pd.to_datetime(first_second_dates_df['SecondLineStartDate'])).dt.days / 30).round()

query4 = """
SELECT 
    c.TreatmentYearMonth,
    COUNT(DISTINCT CASE WHEN r.NEXTLINE_CD38_FLAG = 1 THEN r.PATIENTID END) AS CD38_Patients,
    COUNT(DISTINCT CASE WHEN r.NEXTLINE_CD38_FLAG = 0 THEN r.PATIENTID END) AS Non_CD38_Patients,
    AVG(CASE WHEN r.NEXTLINE_CD38_FLAG = 1 THEN r.MonthsToNextLine END) AS AvgMonthsToNextLine_CD38,
    AVG(CASE WHEN r.NEXTLINE_CD38_FLAG = 0 THEN r.MonthsToNextLine END) AS AvgMonthsToNextLine_Non_CD38
FROM 
    calendar_df c
LEFT JOIN 
    result2 r ON c.TreatmentYearMonth = r.TreatmentYearMonth
GROUP BY 
    c.TreatmentYearMonth
"""
result4 = pds.sqldf(query4, locals())
result4.to_clipboard()
result4

In [ ]:
print(first_second_dates_df['FirstLineStartDate'].isna().any())
print(first_second_dates_df['SecondLineStartDate'].isna().any())
print(result2['MonthsToNextLine'].isna().any())

In [ ]:
merge_data['FIRST_CD38_DATE'] = pd.to_datetime(merge_data['FIRST_CD38_DATE'], errors='coerce')
merge_data['CD38_EXPOSED_FLAG'] = np.where(merge_data['STARTDATE'] > merge_data['FIRST_CD38_DATE'], 1, 0)

In [ ]:
merge_data['']

In [ ]:
pd.set_option('display.max_columns', None)
merge_data.head(17)
merge_data.to_clipboard()

In [ ]:
data = pd.read_csv(r"C:\Users\MichaelDiFelice\Documents\Sanofi\Python\Dashboard\Data\Raw Data\flatiron_newlot_final_v2.csv")

